This script will be used to validate the locations of the measurement stations and the validation stations. If they are located within the area of Amsterdam under review and not in the water, they are valid. Subsequently elevation data is added to the measurement stations in this script to add an additional variable for the analysis. 

Downloading packages, set working directory and disable warnings. When running the script, make sure the working directory is set correctly.

In [1]:
rm(list=ls())
options("rgdal_show_exportToProj4_warnings"="none") # ignore warnings
setwd("~/WUR/Periode_5/DSME/amsterdam_dtm")

In [1]:
if( !("raster" %in% installed.packages()[,1]) )
  install.packages("raster")

if( !("rgdal" %in% installed.packages()[,1]) )
  install.packages("rgdal")

if( !("rjson" %in% installed.packages()[,1]) )
  install.packages("rjson")

if( !("sp" %in% installed.packages()[,1]) )
  install.packages("sp")

In [ ]:
library(sp)
library(raster)
library(rgdal)
library(rjson)

Warning message:
"package 'sp' was built under R version 3.6.3"Warning message:
"package 'raster' was built under R version 3.6.3"

Loading the DSMs and creating a single DSM for the area under review, set the projections for spatial analysis.The file names refer to the tiles from PDOK, make sure you have these files before running the script.

In [ ]:
amsterdam = merge(raster("M_25BZ1.TIF"),raster("M_25BZ2.TIF"), raster("M_25DN1.TIF"),raster('M_25DN2.TIF'),raster("M_25EZ1.TIF"),raster("M_25EZ2.TIF"),
                  raster("M_25GN1.TIF"), raster("M_25GN2.TIF"), raster("M_25GZ1.TIF"), raster("M_25GZ2.TIF"), raster("M_25HN1.TIF"))

CRS_RDnew <- CRS("+init=epsg:28992")
CRS_WGS84 <- CRS("+init=epsg:4326")

Load a shapefile with an outline of Amsterdam

In [ ]:
nld <- raster::getData('GADM', country="NLD", level=2)
ams <- nld[nld[[6]] == "NLD.9.4_1",]

Change working directory and create a dataframe with the coordinates of the measurement stations of the Netatmo dataset, prevent double location from being added.

In [ ]:
setwd("~/WUR/Periode_5/DSME/export-KNMI_Amsterdam_01-05-2016_01-06-2018")

df = as.data.frame(fromJSON(file="0.metadata.json"))$place.location

for (i in 1:562){
  json = fromJSON(file = paste0(sprintf("%01d", i),".metadata.json"))
  if(data.frame(json)$place.location[1] != df[1]){
    if(data.frame(json)$place.location[2] != df[2]){
      df = rbind(df,(as.data.frame(json)$place.location))
    }}}
remove(i)
remove(json)

remove NA values from the dataframe

In [ ]:
for (i in 1:nrow(df)){
  if(is.na(df[i,1])|is.na(df[i,2])){
    df <- df[-c(1),]
    i = i-1
  }
}
remove(i)

set index for measurement stations to keep track of which ones are invalid, set to coordinates and set correct projection

In [ ]:
df = as.data.frame(df)
df$m_station <- seq.int(nrow(df))
coordinates(df) = c(1,2)
crs(df) <- CRS_WGS84
df <- spTransform(df,CRS_RDnew)

Extract the elevation values for each measurement station and remove NA values and save data

In [ ]:
m_elev_dat <- extract(amsterdam,df, small = T,sp=T,fun=mean,buffer=5,na.rm=T)
elev_extract <- as.data.frame(m_elev_dat)

delme <- which(is.na(elev_extract$layer))
if(length(delme) > 0){
  elev_extract<- elev_extract[-delme,]
}
remove(delme)

write.csv(elev_extract,"m_elev.csv",row.names = F)

The next part will load the data from the validation dataset and extract the coordinates to check their location. Again, elevation values will be added as they are not included in the dataset

In [1]:
setwd("~/WUR/Periode_5/DSME/amsterdam_dtm/Meteo-1")

read_excel_allsheets <- function(fname, tibble = F) {
  sheets <- readxl::excel_sheets(fname)
  x <- lapply(sheets, function(X) readxl::read_excel(fname, sheet = X,n_max = 1,col_names = F))
  if(!tibble) x <- lapply(x, as.data.frame)
  names(x) <- sheets
  x
}

Change the rownames and put the data in a dataframe and remove NA values

In [ ]:
first_rows <- read_excel_allsheets("val_dat.xlsx")
names(first_rows) <- c(seq(from=1,to=24))

coor <- data.frame()
for (i in 1:24){ 
  t <- unlist(first_rows[i],use.names = F)
  coor <- rbind(coor,t[14])
  }

delme <- which(is.na(coor))
if(length(delme) > 0){
  coor<- coor[-delme,]
}

remove(delme)
remove(t)
remove(i)

Change the coordinates that are now in a string into the seperate x and y components, transform the data into a spatial points dataframe and set the correct projection

In [ ]:
coor <- as.data.frame(coor)
coordinaten <- matrix(nrow = nrow(coor),ncol=2)

for (i in 1:nrow(coor)){
  crdns <- unlist(strsplit(coor[i,1]," "))
  coordinaten[i,1] <- as.numeric(crdns[1])
  coordinaten[i,2] <- as.numeric(crdns[2])
}
remove(crdns)

coordinaten <- as.data.frame(coordinaten)
coordinates(coordinaten) = c(1,2)
crs(coordinaten) <- CRS_RDnew

Extract the elevation values for the validation measurement stations, visually inspect the locations and save the file

In [ ]:
val_elev <- extract(amsterdam,coordinaten,small = T,fun=mean,buffer=5,na.rm=T)
mapview(ams) + val_elev
write.csv(val_elev,"val_elev.csv")

Looking at the plot, all locations are within the research area and can thus be included in the analysis 

To conclude, the main problem with the current script was the processing time. As the rasters have a high spatial resolution, which was needed to attempt to capture the high spatial variability, the loading and all raster operations take a long time. It was quite a challenge to figure out how to effectively extract the elevation data and validate the location of the stations, as many attempts failed due to memory running out. It would have been better to start with seperate tiles or use lower resultion DSM to find a method to extract and validate. Now, a lot of time was wasted on waiting while the method did not produce the desired results. Working with spatial data is always challenging as the projections were not entirely clear to me in the beginning. Subsequently, the locations were now selected based on the raster file which is a combination of seperate tiles. Thus it encompasses not only the area of Amsterdam, but also some area around it. Using a shapefile of the research area and use this is better to limit the area under review. I was struggeling with finding where and how to download shapefiles of the city so that is something I will do differently in the future.

Relating to the choices made in the current analysis, some 20 measurement stations were ommitted from the dataset as they are located outside of Amsterdam. While this was a choice that improved processing time of the NETATMO dataset by reducing the amount of points, it possibly also ommitted valuable data. The measurement stations located outside of the city are less influenced by buildings, UHI etc. and possible provide more accurate measurements.